In [1]:
import pandas as pd

# Load the datasets
user0_orders = pd.read_csv('user0_orders.csv')
user1_orders = pd.read_csv('user1_orders.csv')
user2_orders = pd.read_csv('user2_orders.csv')
user3_orders = pd.read_csv('user3_orders.csv')

# Combine all user order datasets
all_user_orders = pd.concat([user0_orders, user1_orders, user2_orders, user3_orders])

# Create a user-item matrix with order counts
user_item_matrix = all_user_orders.groupby(['user_id', 'meal_id']).size().unstack(fill_value=0)

# Display the user-item matrix
print(user_item_matrix)

meal_id  0   1   2   3   4   5   6   7   8   9   ...  24  25  26  27  28  29  \
user_id                                          ...                           
0         2   0   2   2   3   1   3   2   3   1  ...   4   0   3   2   3   4   
1         3   4   2   1   2   3   6   1   2   1  ...   1   3   1   1   1   6   
2         4   1   3   3   4   0   3   1   4   4  ...   4   3   2   3   5   2   
3         3   2   1   3   4   3   0   7   2   2  ...   0   8   1   3   3   3   

meal_id  30  31  32  33  
user_id                  
0         3   2   5   4  
1         5   1   1   2  
2         2   0   5   3  
3         4   0   4   2  

[4 rows x 34 columns]


In [2]:
from surprise import Dataset
from surprise import Reader


df = pd.DataFrame(user_item_matrix)
df
# reader = Reader(rating_scale=(1, 5))

# data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)
# movielens = Dataset.load_builtin('ml-100k')

meal_id,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
user_id,,,,,,,,,,,,,,,,,,,,,
0,2,0,2,2,3,1,3,2,3,1,...,4,0,3,2,3,4,3,2,5,4
1,3,4,2,1,2,3,6,1,2,1,...,1,3,1,1,1,6,5,1,1,2
2,4,1,3,3,4,0,3,1,4,4,...,4,3,2,3,5,2,2,0,5,3
3,3,2,1,3,4,3,0,7,2,2,...,0,8,1,3,3,3,4,0,4,2


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity
user_similarity_matrix = pd.DataFrame(cosine_similarity(user_item_matrix), index=user_item_matrix.index, columns=user_item_matrix.index)

# Display the user similarity matrix
print(user_similarity_matrix)

user_id         0         1         2         3
user_id                                        
0        1.000000  0.780033  0.818481  0.748398
1        0.780033  1.000000  0.673856  0.723147
2        0.818481  0.673856  1.000000  0.701571
3        0.748398  0.723147  0.701571  1.000000


In [4]:
# Specify the neighborhood size
neighborhood_size = 5

# Identify top-N similar users for each user
top_similar_users = user_similarity_matrix.apply(lambda row: row.nlargest(neighborhood_size).index.tolist(), axis=1)

# Display the top similar users
print(top_similar_users)

user_id
0    [0, 2, 1, 3]
1    [1, 0, 3, 2]
2    [2, 0, 3, 1]
3    [3, 0, 1, 2]
dtype: object


In [5]:
# Function to generate recommendations for a user
def generate_recommendations(user_id):
    similar_users = top_similar_users.loc[user_id]
    user_orders = user_item_matrix.loc[user_id]

    # Identify food items not ordered by the target user
    not_ordered_items = user_orders[user_orders == 0].index

    # Aggregate food items ordered by similar users
    similar_users_orders = user_item_matrix.loc[similar_users].sum()

    # Exclude items already ordered by the target user
    recommended_items = similar_users_orders[not_ordered_items]

    # Sort recommended items by order count in descending order
    recommended_items = recommended_items.sort_values(ascending=False)

    return recommended_items

# Example: Generate recommendations for user 0
user_id_to_recommend = 0
recommendations = generate_recommendations(user_id_to_recommend)

# Display the recommendations
print(f"Recommendations for User {user_id_to_recommend}:\n{recommendations}")

Recommendations for User 0:
meal_id
25    14
1      7
21     6
dtype: int64


In [15]:
food_data = pd.read_csv('food_data_1.csv')

# Function to display recommendations when a user logs in
def display_recommendations_on_login(user_id):
    recommendations = generate_recommendations(user_id)

    # Assuming you have a user interface, you can display the recommendations there
    print(f"Hello User {user_id}! Here are your personalized recommendations:")
    for food_id, order_count in recommendations.items():
        print(f"Meal: {food_data.loc[food_id]['name']}, Order Count: {order_count}")
    print('\n')
# Example: Display recommendations when User 0 logs in
user_id_logged_in = 0
display_recommendations_on_login(user_id_logged_in)

Hello User 0! Here are your personalized recommendations:
Meal: Kavurmalı Pilav, Order Count: 14
Meal: İskender, Order Count: 7
Meal: Ispanaklı Karnabahar, Order Count: 6




In [16]:
user_ids = [0, 1, 2, 3]
for user in user_ids:
    display_recommendations_on_login(user)

Hello User 0! Here are your personalized recommendations:
Meal: Kavurmalı Pilav, Order Count: 14
Meal: İskender, Order Count: 7
Meal: Ispanaklı Karnabahar, Order Count: 6


Hello User 1! Here are your personalized recommendations:
Meal: İmam Bayıldı, Order Count: 6


Hello User 2! Here are your personalized recommendations:
Meal: Havuç Tarator, Order Count: 13
Meal: Ali Nazik Kebabı, Order Count: 7
Meal: Künefe, Order Count: 3


Hello User 3! Here are your personalized recommendations:
Meal: Baklava, Order Count: 12
Meal: Tavuk Döner, Order Count: 11
Meal: Karnabahar Kızartması, Order Count: 9
Meal: Künefe, Order Count: 3




In [29]:
import requests

purchase_feedback = {"user_id": 1, "food_id": 2}
response = requests.post('http://localhost:5002/feedback', json=purchase_feedback)

print(response.json())

{'message': 'Feedback received and matrix updated successfully'}


In [59]:
from sklearn.metrics import precision_score, recall_score
import numpy as np

# Assuming you have user_item_matrix as your training data
# You can copy it to create a test_user_item_matrix and randomly mask some entries as interactions
test_user_item_matrix = user_item_matrix.copy()

# Mask 20% of the entries as interactions (1s)
mask = np.random.choice([0, 1], size=test_user_item_matrix.shape, p=[0.8, 0.2])
test_user_item_matrix = test_user_item_matrix * mask

from sklearn.metrics import precision_score, recall_score

# Function to evaluate the recommendation system
def evaluate_recommendations(user_item_matrix, test_user_item_matrix, top_n=5):
    precision_scores = []
    recall_scores = []

    for user_id in user_item_matrix.index:
        # Generate recommendations for the user
        recommendations = generate_recommendations(user_id)[:top_n]

        # Get the actual items the user interacted with in the test set
        actual_items = test_user_item_matrix.loc[user_id, test_user_item_matrix.loc[user_id] > 0].index

        # Ensure both arrays have the same length
        min_len = min(len(actual_items), len(recommendations))
        actual_items = actual_items[:min_len]
        recommendations = recommendations[:min_len]

        # Evaluate precision and recall
        precision = precision_score(actual_items, recommendations, labels=actual_items, average='micro', zero_division=0)
        recall = recall_score(actual_items, recommendations, labels=actual_items, average='micro', zero_division=0)

        precision_scores.append(precision)
        recall_scores.append(recall)

    # Calculate average precision and recall
    avg_precision = sum(precision_scores) / len(precision_scores)
    avg_recall = sum(recall_scores) / len(recall_scores)

    return avg_precision, avg_recall

# Example: Evaluate the recommendation system
avg_precision, avg_recall = evaluate_recommendations(user_item_matrix, test_user_item_matrix, top_n=5)
print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")


Average Precision: 0.25
Average Recall: 0.25
